# Strata Scratch

## Intro to Pandas 

- Primary objects in Pandas are called DataFrames
- DataFrames are like Excel/database tables 
    - Contain rows and columns of data
    - Columns have names
    - Rows have index values 
- Pandas has easy functions for importing and exporting data 
    - CSV files
    - Excel spreadsheets 
    - SQL queries 


Import the pandas package

In [ ]:
import pandas as pd
pd.__version__

Create a simple DataFrame

- syntax: pd.DataFrame({column1 : value, column2 : value})

In [ ]:
df = pd.DataFrame({'name':['Bob','Jen','Tim'],
                   'age':[20,30,40],
                   'pet':['cat', 'dog', 'bird']})

df

View the column names and index values

In [ ]:
print(df.columns)
print(df.index)

Select a column by name in 2 different ways

In [ ]:
print(df['name'])
print(df.name)

Select multiple columns

In [ ]:
print(df[['name','pet']])

Select a row by index

In [ ]:
print(df.iloc[0])

### Sort Function

- pandas.pydata.org
- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html

Sort the data by pet

In [ ]:
df.sort_values('pet',inplace=True, ascending=True)

### Indexing with DataFrames

- Row and column are the relative position of the data cells you want 
- To select multiple rows or columns, use a colon to separate the start and end values 
- Colon with no value returns all rows or columns 

View the index after the sort

In [ ]:
print(df)

Difference between loc and iloc

In [ ]:
print(df.loc[0]) #index based
print(df.iloc[0]) #relative position based indexing

Use iloc to select all rows of a column

In [ ]:
print(df.iloc[:,2])

In [ ]:
df

Use iloc to select the last row

In [ ]:
df.iloc[-1,:]

# Exercises

In [ ]:
sales = [100,130,119,92,35]
customer_account = ['B100','J101','X102','P103','R104']
city = ['BOS','LA','NYC','SF','CHI']

Create a DataFrame with the data above

In [ ]:
df = pd.DataFrame({'sales':sales, 'customer':customer_account, 
                   'city':city})

In [ ]:
print(df)

What is the name of the first column?

In [ ]:
print(df.columns[0])
print(df.city)
print(df['city'])

Sort the DataFrame by city in descending order (check the documentation for sort)

In [ ]:
df.sort_values('city',ascending=False,inplace=True)
print(df)

Which customer is in the last row of the DataFrame?

In [ ]:
print(df.iloc[-1,1])
print(df.customer.iloc[-1])

## Bonus
Reorder the columns with customer in the first column

In [ ]:
df = df[['customer','city','sales']]
print(df)

Rename a column

In [ ]:
df.columns = ['customer', 'city_name', 'sales_amount']
print(df)

In [ ]:
df.rename(columns={'customer':'customer_account'},inplace=True)
print(df)

## Importing and Exporting Data with Pandas

#### Pandas has easy to use functions for importing and exporting different data types: 
- CSV Files 
- Excel Worksheets
- Queries from Databases

### Reading and Writing CSV Files

- Import data from CSV files
- Investigate data
    - View samples of the data
    - Evaluate summary statistics
- Filter and slice the data for analysis

__ Exploring Titanic Data Set with Pandas__

In [ ]:
import pandas as pd

Load the titanic.csv file as a DataFrame

In [ ]:
data = pd.read_csv('titanic.csv')

Investigate the first few rows of data

In [ ]:
data.head()

Investigate the last 10 rows of data

In [ ]:
data.tail(10)

Investigate the data types in the DataFrame

In [ ]:
data.info()

Get some summary statistics

In [ ]:
data.describe().T

Filter the data for men

In [ ]:
data[data.Sex=='male']

Filter the ages for the men

In [ ]:
data.Age[data.Sex=='male']

How many men and women were on the Titanic?

In [ ]:
print(data.Sex[data.Sex=='male'].count())
print(data.Sex[data.Sex=='female'].count())

What was the survival rate for adult men (age>=18)

In [ ]:
data.Survived[(data.Sex=='male')&(data.Age>=18)].mean()

What was the survival rate for women and children?

In [ ]:
data.Survived[(data.Sex=='female')|(data.Age<18)].mean()

Use groupby to compare the survival rates of men and women

In [ ]:
data.groupby('Sex')['Survived'].mean()

Create a DataFrame with groupby and view the index

In [ ]:
new = data.groupby(['Sex','Pclass'])['Survived','Age'].mean()
print(new.index)
print(new)

Reset the index 

In [ ]:
new.reset_index(inplace=True)

View again

In [ ]:
print(new.index)
print(new)

# Exercises

What was the average age of the survivors?

In [ ]:
print(data.Age[data.Survived==1].mean())

What was the combined survival rate of both children (age less than 18) and seniors (age greater than 60)?

In [ ]:
print(data.Survived[(data.Age<18)|(data.Age>=60)].mean())

Group by pClass and investigate average survival rate, age and fare

In [ ]:
# data.groupby('Pclass')['Age','Survived','Fare'].mean()
data.groupby(['Sex','Pclass'])['Age','Survived','Fare'].mean()

Create a CSV or Excel file with the names and ages of the surivors and another CSV or Excel file with the names and ages of the deceased. Please refer to documention (to_csv and/or to_excel functions) to complete the exercise.

In [ ]:
writer = pd.ExcelWriter('titanic_survivors.xlsx')
survived = data[['Name','Age']][data.Survived==1]
survived.to_excel(writer,'Survivors')
data[['Name','Age']][data.Survived==0].to_excel(writer,
                                                'Deceased',
                                                index=False)

writer.save()

## Making Database calls with Strata Scratch

- Strata Scratch has a postgres backend, therefore the psycopgy2 library must be installed and imported before you can make database calls directly in python. 
- Otherwise, you can always use SQL LAB in Strata Scratch and export your data pulls as an CSV file, then import the file using pandas.
- Installing psycopg2 for Anaconda: https://anaconda.org/anaconda/psycopg2

Here's some code to make database calls to Strata Scratch

In [ ]:
import psycopg2 as ps

In [ ]:
host_name = 'db-strata.stratascratch.com'
dbname = 'db_strata'
user_name = '' #enter username and password from profile tab in Strata Scratch
pwd = ''
port = '5432'

In [ ]:
try:
    conn = ps.connect(host=host_name,database=dbname,user=user_name,password=pwd,port=port)
except ps.OperationalError as e:
    raise e
else:
    print('Connected!')

In [ ]:
#Make the database call
cur = conn.cursor()
cur.execute(""" 
            SELECT *  FROM datasets.titanic; 
            """)
df = cur.fetchall()
colnames = [desc[0] for desc in cur.description] #grab the column names
conn.commit()

#create the dataframe
df=pd.DataFrame(df)
df.columns = colnames

#close the connection
cur.close()